In [19]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
import glob
import imghdr
import itertools
import tensorflow_hub as hub

from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from __future__ import absolute_import, division, print_function, unicode_literals

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF version: 2.9.1
Hub version: 0.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [3]:
tf.__version__

'2.9.1'

In [6]:
data_dir = pathlib.Path("C:/Users/Ru/Desktop/Aruviyn/Year 3 Sem 2/FYP_System/sneaker_dataset/images/train2")

In [7]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 27847 images belonging to 574 classes.
Found 6680 images belonging to 574 classes.


In [8]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 574))

In [9]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Air Force 1 Flyknit 2 White Pure Platinum': 0, 'Air Force 1 Flyleather Steve Harrington Earth Day (2019)': 1, 'Air Force 1 Foamposite Cup University Red': 2, 'Air Force 1 High Emotionally Unavailable': 3, 'Air Force 1 High Flax (2018)': 4, 'Air Force 1 LX Vandalised White (W)': 5, 'Air Force 1 Low 3x1 Denim Raw Indigo': 6, 'Air Force 1 Low A Cold Wall Black': 7, 'Air Force 1 Low Carhartt WIP Ale Brown': 8, 'Air Force 1 Low De Lo Mio': 9, 'Air Force 1 Low Devin Booker': 10, 'Air Force 1 Low Equality': 11, 'Air Force 1 Low Noise Cancelling Pack Odell Beckham Jr': 12, 'Air Force 1 Low Off-White Black White': 13, 'Air Force 1 Low Off-White MCA University Blue': 14, 'Air Force 1 Low Off-White Volt': 15, 'Air Force 1 Low Retro Ivory Snake': 16, 'Air Force 1 Low Roc-A-Fella (AF100)': 17, 'Air Force 1 Low Supreme x Comme des Garcons (2018)': 18, 'Air Force 1 Low Travis Scott (AF100)': 19, 'Air Force 1 Low Travis Scott Sail': 20, 'Air Force 1 Low Utility Black White': 21, 'Air Force 1 Low VLO

In [11]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

9406464/9406464 [==============================] - 7s 1us/step


In [12]:
base_model.trainable = False

In [13]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 32)          368672    
                                                                 
 dropout (Dropout)           (None, 5, 5, 32)          0         
                                                                 
 global_average_pooling2d (G  (None, 32)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 2,626,821
Trainable params: 368,837
Non-tr

In [16]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


In [18]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Users\Ru\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Ru\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Ru\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Ru\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Ru\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Ru\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Ru\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self